# Scikit-Learn Linear Regression
Using SALES_VIEW from DWC. This view has 6,291,450 records

## Install fedml-azure package

In [2]:
pip install fedml-azure --force-reinstall

Processing ./fedml_azure_test-2.0.0-py3-none-any.whl
  Using cached ruamel.yaml-0.17.21-py3-none-any.whl (109 kB)
  Using cached hdbcli-2.12.20-cp34-abi3-manylinux1_x86_64.whl (11.7 MB)
  Using cached ruamel.yaml.clib-0.2.6-cp38-cp38-manylinux1_x86_64.whl (570 kB)
  Attempting uninstall: ruamel.yaml.clib
    Found existing installation: ruamel.yaml.clib 0.2.6
    Uninstalling ruamel.yaml.clib-0.2.6:
      Successfully uninstalled ruamel.yaml.clib-0.2.6
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.17.21
    Uninstalling ruamel.yaml-0.17.21:
      Successfully uninstalled ruamel.yaml-0.17.21
  Attempting uninstall: hdbcli
    Found existing installation: hdbcli 2.12.20
    Uninstalling hdbcli-2.12.20:
      Successfully uninstalled hdbcli-2.12.20
  Attempting uninstall: fedml-azure-test
    Found existing installation: fedml-azure-test 2.0.0
    Uninstalling fedml-azure-test-2.0.0:
      Successfully uninstalled fedml-azure-test-2.0.0
Note: you may n

## Import the libraries needed in this notebook

In [3]:
from fedml_azure import create_workspace
from fedml_azure import DbConnection
from fedml_azure import create_compute
from fedml_azure import create_environment
from fedml_azure import DwcAzureTrain
from fedml_azure import deploy
from fedml_azure import predict
from fedml_azure import register_model

## Set up

### Initialize the workspace

The create_workspace method takes a dictionary as input for parameter workspace_args.

Before running the below cell, ensure that you have a workspace and replace the subscription_id, 
resource_group, and workspace_name with your information.
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?tabs=python 

Refer the documentation on the ‘create_workspace’ method and parameters (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#create_workspace).


In [4]:
workspace=create_workspace(workspace_args={
                                            "subscription_id": '<subscription-id>',
                                            "resource_group": '<resource-group>',
                                            "workspace_name": '<workspace_name>'
                                            }
)

2022-04-01 22:34:37,712: fedml_azure.logger INFO: Getting existing Workspace


### Create a Compute target

The create_compute method takes the workspace, a compute_type, and compute_args as parameters.The following code creates a Compute Cluster with the name 'cluster' for training.

Refer the documentation on the ‘create_compute’ method and parameters (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#create_compute).


In [5]:
compute=create_compute(workspace=workspace,
                   compute_type='AmlComputeCluster',
                   compute_args={'vm_size':'Standard_D12_v2',
                                'vm_priority':'lowpriority',
                                'compute_name':'cpu-cluster',
                                'min_nodes':0,
                                'max_nodes':4,
                                'idle_seconds_before_scaledown':1700
                                }
                )

2022-04-01 22:34:39,991: fedml_azure.logger INFO: Creating Compute_target.
2022-04-01 22:34:41,715: fedml_azure.logger INFO: Creating a new compute target...
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
2022-04-01 22:34:48,236: fedml_azure.logger INFO: {'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2022-04-01T22:34:42.607000+00:00', 'errors': None, 'creationTime': '2022-04-01T22:34:42.197041+00:00', 'modifiedTime': '2022-04-01T22:34:45.987396+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1700S'}, 'vmPrior

### Create an Environment

The create_environment method takes the workspace, environment_type, and environment_args as parameters.

Pass 'fedml-azure' as a pip package and to use scikit-learn, you must pass the name to conda_packages as well.

Refer the documentation on the ‘create_environment’ method and parameters (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#create_environment).
`

In [8]:
environment=create_environment(workspace=workspace,
                           environment_type='CondaPackageEnvironment',
                           environment_args={'name':'regression-sklearn',
                                             'pip_packages':['joblib','fedml-azure'],
                                             'conda_packages':['scikit-learn']})


2022-04-01 22:35:04,155: fedml_azure.logger INFO: Creating Environment.


## Now, lets train the model

### Creating a Training object and setting the workspace, compute target, and environment.

Before running the below cell, ensure that you have a workspace and replace the subscription_id, resource_group, and workspace_name with your information.

Pass the 'fedml_azure' as a pip package and to use scikit-learn, you must pass the name to conda_packages as well.

Refer the documentation on the ‘DwcAzureTrain’ class (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#dwcazuretrain-class).

In [9]:
train=DwcAzureTrain(workspace=workspace,
                    environment=environment,
                    experiment_args={'name':'regression-experiment'},
                    compute=compute)

2022-04-01 22:35:05,374: fedml_azure.logger INFO: Assigning Workspace.
2022-04-01 22:35:05,375: fedml_azure.logger INFO: Creating Experiment
2022-04-01 22:35:05,521: fedml_azure.logger INFO: Assigning compute.
2022-04-01 22:35:05,522: fedml_azure.logger INFO: Assigning Environment.


### Then, we need to generate the run config. This is needed to package the configuration specified so we can submit a job for training. 

Before running the following cell, you should have a config.json file with the specified values to allow you to access to DWC. Provide this file path to config_file_path in the below cell.

You should also have the follow view SALES_VIEW created in your DWC. To gather this data, please refer to https://eforexcel.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/

Please note the 2M records data was downloaded and duplicated 3 times to represent a large dataset in DWC.

Refer the documentation on the ‘generate_run_config’ method and parameters https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#generate_run_config). 

In [10]:
src=train.generate_run_config(config_file_path='dwc_configs/config.json',
                          config_args={
                                          'source_directory':'Scikit-Learn-Linear-Regression',
                                          'script':'train_script.py',
                                          'arguments':[
                                                        '--model_file_name','regression.pkl',
                                                        '--table_name', 'SALES_VIEW'
                                                      ]
                                          }
                            )

2022-04-01 22:35:07,068: fedml_azure.logger INFO: Generating script run config.
2022-04-01 22:35:07,069: fedml_azure.logger INFO: Copying config file for db connection to script_directory Scikit-Learn-Linear-Regression


### Submit the training job with the option to download the model outputs

Refer the documentation on ‘submit_run’ method and parameters (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#submit_run)

In [11]:
run=train.submit_run(src)

2022-04-01 22:35:09,654: fedml_azure.logger INFO: Submitting training run.
RunId: regression-experiment_1648852509_6fa36175
Web View: https://ml.azure.com/runs/regression-experiment_1648852509_6fa36175?wsid=/subscriptions/cb97564e-cea8-45a4-9c5c-a3357e8f7ee4/resourcegroups/AI_Strategy_AzureML_Resource/workspaces/FedMLAzure&tid=42f7676c-f455-423c-82f6-dc2d99791af7

Execution Summary
RunId: regression-experiment_1648852509_6fa36175
Web View: https://ml.azure.com/runs/regression-experiment_1648852509_6fa36175?wsid=/subscriptions/cb97564e-cea8-45a4-9c5c-a3357e8f7ee4/resourcegroups/AI_Strategy_AzureML_Resource/workspaces/FedMLAzure&tid=42f7676c-f455-423c-82f6-dc2d99791af7



### Register the model

Pass ‘outputs/model_file_name.pkl’ to 'model_path' key of model_args ,where ‘model_file_name’ is the name of the .pkl model file specified in the previous step. 

Provide the desired model name to ‘model_name’ key of model_args in the below cell. The 'is_sklearn_model' flag specifies if a scikit learn model is being registered.

Refer the documentation on ‘register_model’ method and parameters (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#register_model)

In [13]:
model=train.register_model(run=run,
                           model_args={'model_name':'sklearn_linReg_model',
                                       'model_path':'outputs/regression.pkl'},
                            resource_config_args={'cpu':1, 'memory_in_gb':0.5},
                            is_sklearn_model=True
                           )

2022-04-01 22:43:37,144: fedml_azure.logger INFO: Registering the model.
2022-04-01 22:43:37,145: fedml_azure.logger INFO: Configuring parameters for sklearn model.


### Register the model without the training run with the model file. This is helpful if you need to deploy models with just the model file. This step is optional

 #### For this sample use case, we download the model generated from the training run. This step is optional

In [14]:
train.download_files(run)

2022-04-01 22:43:44,348: fedml_azure.logger INFO: Downloading the required files.
2022-04-01 22:43:44,349: fedml_azure.logger INFO: Downloading the files to outputs/regression-experiment/regression-experiment_1648852509_6fa36175.


We now use the model file to register the model.  This step is optional

Refer the documentation on 'register_model' for more details (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#register_model_for_deploy)

In [69]:
model=register_model(model_args={'workspace':workspace,
                                'model_name':'regression_model',
                                'model_path':'outputs/regression-experiment/regression-experiment_1648852509_6fa36175/outputs/regression.pkl'},
                                resource_config_args={'cpu':1, 'memory_in_gb':0.5},
                                is_sklearn_model=True
                                )

2022-04-01 23:46:43,621: fedml_azure.logger INFO: Registering the model
2022-04-01 23:46:43,622: fedml_azure.logger INFO: Configuring parameters for sklearn model
Registering model regression_model


### Get the test data from SAP DWC

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from fedml_azure import DbConnection
import pandas as pd
import numpy as np
import json 

with open('Scikit-Learn-Linear-Regression/config.json', 'r') as f:
    config = json.load(f)


label_column = 'totalprofit'

def get_data(table_name):
    dwc = DbConnection(url='Scikit-Learn-Linear-Regression/config.json')
    schema=config['schema']
    query=f'SELECT TOP 100 * FROM "{schema}"."{table_name}"'
    data = dwc.execute_query(query)
    data = pd.DataFrame(data[0], columns=data[1])
    data=data[['unitssold', 'unitprice', 'unitcost','totalrevenue', 'totalcost','totalprofit']]
    return data

data = get_data('sales_view_athena') 
data=data.dropna()#getting data from DWC
y = data[label_column]

data.drop(label_column, axis=1, inplace=True)

print(np.shape(data), np.shape(y))

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3)

(99, 5) (99,)


In [20]:
test_data = json.dumps({
    'data': X_test.values.tolist()
})

 ### Deploy the model as a Webservice to Azure Compute Instance

Refer the documentation on 'deploy' for more details (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#deploy)

In [22]:
aci_endpoint,_,aci_service=deploy(compute_type='ACI',
                                 inference_config_args={'entry_script':'Scikit-Learn-Linear-Regression/score.py', 'environment':environment},
                                 deploy_config_args={'cpu_cores':1, 'memory_gb':0.5},
                                 deploy_args={'workspace':workspace,'name':'aciwebservice','models':[model]}
                                )

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-01 22:46:35+00:00 Creating Container Registry if not exists.
2022-04-01 22:46:36+00:00 Use the existing image.
2022-04-01 22:46:39+00:00 Generating deployment configuration..
2022-04-01 22:46:54+00:00 Submitting deployment to compute..
2022-04-01 22:47:20+00:00 Checking the status of deployment aciwebservice..
2022-04-01 22:49:50+00:00 Checking the status of inference endpoint aciwebservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2022-04-01 22:49:56,097: fedml_azure.logger INFO: The service endpoint url is http://ea42dbb9-1f9c-47d9-a696-c3a4faa561d4.eastus.azurecontainer.io/score.


### Inference the ACI endpoint using the test data

Refer the documentation on 'predict' for more details (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#predict)

#### Inference the ACI endpoint using the webservice object

In [56]:
result=predict(service=aci_service,data=test_data)
result

2022-04-01 23:43:57,076: fedml_azure.logger INFO: Using the webservice for inferencing. It can be used for compute types ACI, AKS and Local. 


'{"result": [574814.8800000149, 57771.99999999383, 780920.8500000031, 16473.599999995786, 82823.04000000824, 763463.1699999849, 18513.62000002228, 373994.3699999871, 197197.1400000148, 926422.4999999545, 73706.88000001313, 127593.9600000171, 362171.20999998716, 780256.8399999604, 17612.28000002297, 291262.399999983, 18189.599999995684, 367783.8000000103, 1391133.869999982, 115376.24000002054, 14069.580000025686, 82951.02000002284, 55884.39999999397, 1247602.4999999478, 474321.24999996333, 222082.56000002025, 43986.79999999452, 114360.98000000778, 119000.05000001748, 1142688.7499999497]}'

#### Inference the ACI endpoint using the endpoint_url

In [57]:
result=predict(endpoint_url=aci_endpoint,compute_type='ACI',data=test_data)
result

2022-04-01 23:43:59,428: fedml_azure.logger INFO: Using the parameters 'endpoint_url' and 'compute_type' for inferencing.


{'result': [574814.8800000149,
  57771.99999999383,
  780920.8500000031,
  16473.599999995786,
  82823.04000000824,
  763463.1699999849,
  18513.62000002228,
  373994.3699999871,
  197197.1400000148,
  926422.4999999545,
  73706.88000001313,
  127593.9600000171,
  362171.20999998716,
  780256.8399999604,
  17612.28000002297,
  291262.399999983,
  18189.599999995684,
  367783.8000000103,
  1391133.869999982,
  115376.24000002054,
  14069.580000025686,
  82951.02000002284,
  55884.39999999397,
  1247602.4999999478,
  474321.24999996333,
  222082.56000002025,
  43986.79999999452,
  114360.98000000778,
  119000.05000001748,
  1142688.7499999497]}

 ### Deploy the model as a Webservice Locally

Refer the documentation on 'deploy' for more details (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#deploy)

In [31]:
local_endpoint,_,local_service=deploy(compute_type='local',
                                     inference_config_args={'entry_script':'Scikit-Learn-Linear-Regression/score.py', 'environment':environment},
                                    deploy_config_args={'port':8500},
                                    deploy_args={'workspace':workspace,'name':'localregservice','models':[model]}
                                    )

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry d531eb48a65445d3b0ed1b6002c65026.azurecr.io
Logging into Docker registry d531eb48a65445d3b0ed1b6002c65026.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM d531eb48a65445d3b0ed1b6002c65026.azurecr.io/azureml/azureml_acb65c5118a4b8978e42eee54f084834
 ---> 89b6a60fe59f
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 9fccc8340243
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImNiOTc1NjRlLWNlYTgtNDVhNC05YzVjLWEzMzU3ZThmN2VlNCIsInJlc291cmNlR3JvdXBOYW1lIjoiYWlfc3RyYXRlZ3lfYXp1cmVtbF9yZXNvdXJjZSIsImFjY291bnROYW1lIjoiZmVkbWxhenVyZSIsIndvcmtzcGFjZUlkIjoiMDJmMTFhOTEtZDQ2Ny00M2FiLWI3OTEtZTBjODdhM2YwYjI1In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 5b5ac626f62e
 ---> 641877ee9fe0
Step 4/5 : RUN mv '/var/azureml-app/tmpqme0v06i.py' /var/azureml-app/main.py
 ---> Ru

### Inference the Local endpoint using the test data

Refer the documentation on 'predict' for more details (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#predict)

#### Inference the Local endpoint using the webservice object

In [54]:
result=predict(service=local_service,data=test_data)
result

2022-04-01 23:43:01,383: fedml_azure.logger INFO: Using the webservice for inferencing. It can be used for compute types ACI, AKS and Local. 


'{"result": [574814.8800000149, 57771.99999999383, 780920.8500000031, 16473.599999995786, 82823.04000000824, 763463.1699999849, 18513.62000002228, 373994.3699999871, 197197.1400000148, 926422.4999999545, 73706.88000001313, 127593.9600000171, 362171.20999998716, 780256.8399999604, 17612.28000002297, 291262.399999983, 18189.599999995684, 367783.8000000103, 1391133.869999982, 115376.24000002054, 14069.580000025686, 82951.02000002284, 55884.39999999397, 1247602.4999999478, 474321.24999996333, 222082.56000002025, 43986.79999999452, 114360.98000000778, 119000.05000001748, 1142688.7499999497]}'

#### Inference the Local endpoint using the endpoint_url

In [55]:
result=predict(endpoint_url=local_endpoint,data=test_data,compute_type='Local')
result

2022-04-01 23:43:04,486: fedml_azure.logger INFO: Using the parameters 'endpoint_url' and 'compute_type' for inferencing.


{'result': [574814.8800000149,
  57771.99999999383,
  780920.8500000031,
  16473.599999995786,
  82823.04000000824,
  763463.1699999849,
  18513.62000002228,
  373994.3699999871,
  197197.1400000148,
  926422.4999999545,
  73706.88000001313,
  127593.9600000171,
  362171.20999998716,
  780256.8399999604,
  17612.28000002297,
  291262.399999983,
  18189.599999995684,
  367783.8000000103,
  1391133.869999982,
  115376.24000002054,
  14069.580000025686,
  82951.02000002284,
  55884.39999999397,
  1247602.4999999478,
  474321.24999996333,
  222082.56000002025,
  43986.79999999452,
  114360.98000000778,
  119000.05000001748,
  1142688.7499999497]}

 ### Deploy the model as a Webservice to Azure Kubernetes Service

#### Create Azure Kubernetes Service Compute

Refer the documentation on the ‘create_compute’ method and parameters (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#create_compute).

In [26]:
aks=create_compute(workspace=workspace,
                   compute_type='AKS',
                   compute_args={'compute_name':'aks1'})

2022-04-01 22:56:15,394: fedml_azure.logger INFO: Creating Compute_target.
2022-04-01 23:11:54,760: fedml_azure.logger INFO: Found existing cluster, use it.


 #### Deploy the model as a Webservice to Azure Kubernetes Service

Refer the documentation on 'deploy' for more details (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#deploy)

In [28]:
aks_endpoint,api_key,aks_service=deploy(compute_type='AKS',
                                        inference_config_args={'entry_script':'Scikit-Learn-Linear-Regression/score.py', 'environment':environment},
                                        deploy_args={'workspace':workspace,'name':'akswebservice','models':[model],'deployment_target':aks}
                                        )

2022-04-01 23:14:43,690: fedml_azure.logger INFO: The parameter 'deploy_config_args' is not specified. An empty configuration object will be used based on the desired target.
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-01 23:14:49+00:00 Creating Container Registry if not exists.
2022-04-01 23:14:49+00:00 Registering the environment.
2022-04-01 23:14:50+00:00 Use the existing image.
2022-04-01 23:14:51+00:00 Creating resources in AKS.
2022-04-01 23:14:52+00:00 Submitting deployment to compute.
2022-04-01 23:14:52+00:00 Checking the status of deployment akswebservice..
2022-04-01 23:15:02+00:00 Checking the status of inference endpoint akswebservice.
Succeeded
AKS service creation operation finished, operation "Succeeded"
2022-04-01 23:15:06,494: fedml_azure.logger INFO: The service endpoint url is http://20.121.80.180:80/api/v1/service/a

### Inference the AKS endpoint using the test data

Refer the documentation on 'predict' for more details (https://github.com/SAP-samples/data-warehouse-cloud-fedml/blob/main/Azure/docs/fedml_azure.md#predict)

#### Inference the AKS endpoint using the webservice object

In [58]:
result=predict(service=aks_service,data=test_data)
result

2022-04-01 23:44:09,517: fedml_azure.logger INFO: Using the webservice for inferencing. It can be used for compute types ACI, AKS and Local. 


'{"result": [574814.8800000149, 57771.99999999383, 780920.8500000031, 16473.599999995786, 82823.04000000824, 763463.1699999849, 18513.62000002228, 373994.3699999871, 197197.1400000148, 926422.4999999545, 73706.88000001313, 127593.9600000171, 362171.20999998716, 780256.8399999604, 17612.28000002297, 291262.399999983, 18189.599999995684, 367783.8000000103, 1391133.869999982, 115376.24000002054, 14069.580000025686, 82951.02000002284, 55884.39999999397, 1247602.4999999478, 474321.24999996333, 222082.56000002025, 43986.79999999452, 114360.98000000778, 119000.05000001748, 1142688.7499999497]}'

#### Inference the AKS endpoint using the endpoint_url

In [59]:
result=predict(endpoint_url=aks_endpoint,data=test_data,compute_type='aks',api_key=api_key)
result

2022-04-01 23:44:11,385: fedml_azure.logger INFO: Using the parameters 'endpoint_url' and 'compute_type' for inferencing.


{'result': [574814.8800000149,
  57771.99999999383,
  780920.8500000031,
  16473.599999995786,
  82823.04000000824,
  763463.1699999849,
  18513.62000002228,
  373994.3699999871,
  197197.1400000148,
  926422.4999999545,
  73706.88000001313,
  127593.9600000171,
  362171.20999998716,
  780256.8399999604,
  17612.28000002297,
  291262.399999983,
  18189.599999995684,
  367783.8000000103,
  1391133.869999982,
  115376.24000002054,
  14069.580000025686,
  82951.02000002284,
  55884.39999999397,
  1247602.4999999478,
  474321.24999996333,
  222082.56000002025,
  43986.79999999452,
  114360.98000000778,
  119000.05000001748,
  1142688.7499999497]}

### Write the result back to SAP DWC

#### Create table in SAP DWC

In [36]:
from fedml_azure import DbConnection
dwc = DbConnection(url='Scikit-Learn-Linear-Regression/config.json')

In [67]:
dwc.create_table("CREATE TABLE LINEAR_SALES_VIEW (unitssold FLOAT,unitprice FLOAT,unitcost FLOAT,totalrevenue FLOAT,totalcost FLOAT,result FLOAT)")

2022-04-01 23:45:53,385: fedml_azure.logger INFO: creating table...
2022-04-01 23:45:53,386: fedml_azure.logger INFO: CREATE TABLE LINEAR_SALES_VIEW (unitssold FLOAT,unitprice FLOAT,unitcost FLOAT,totalrevenue FLOAT,totalcost FLOAT,result FLOAT, INSERTED_AT TIMESTAMP NOT NULL)


#### Storing the result in the dataframe

In [60]:
import pandas as pd
result_df=pd.DataFrame(result['result'])
result_df.rename( columns={0:'result'}, inplace=True )

In [62]:
X_test['result']=result_df['result'].values
X_test

,unitssold,unitprice,unitcost,totalrevenue,totalcost,result
35,7827.0,109.28,35.84,855334.56,280519.68,574814.88
33,1010.0,421.89,364.69,426108.90,368336.90,57772.00
84,8815.0,205.70,117.11,1813245.50,1032324.65,780920.85
44,288.0,421.89,364.69,121504.32,105030.72,16473.60
60,864.0,255.28,159.42,220561.92,137738.88,82823.04
34,4391.0,437.20,263.33,1919745.20,1156282.03,763463.17
70,7682.0,9.33,6.92,71673.06,53159.44,18513.62
29,2151.0,437.20,263.33,940417.20,566422.83,373994.37
40,7869.0,81.73,56.67,643133.37,445936.23,197197.14
81,7338.0,651.21,524.96,4778578.98,3852156.48,926422.50


 #### Inserting the data into table

In [68]:
dwc.insert_into_table('LINEAR_SALES_VIEW',X_test)

2022-04-01 23:45:57,111: fedml_azure.logger INFO: inserting into table...
2022-04-01 23:45:57,112: fedml_azure.logger INFO: INSERT INTO LINEAR_SALES_VIEW (unitssold, unitprice, unitcost, totalrevenue, totalcost, result, INSERTED_AT) VALUES (:unitssold, :unitprice, :unitcost, :totalrevenue, :totalcost, :result, :INSERTED_AT)
